In [2]:
import numpy as np
import torch
import torch.nn as nn
from pathlib import Path

data_dir = Path("data")

# 1) Load raw + projected embeddings
raw  = np.load(data_dir / "combined_raw_embeddings.npy")       # (N, D_in)
proj = np.load(data_dir / "combined_projected_embeddings.npy") # (N, D_out)

# 2) Rebuild the decoder (must match training)
D_in, H, D_out = raw.shape[1], 512, proj.shape[1]
decoder = nn.Sequential(
    nn.Linear(D_out, H),
    nn.GELU(),
    nn.Linear(H, D_in)
)

# 3) Load trained weights
state = torch.load(data_dir / "decoder.pth", map_location="cpu")
decoder.load_state_dict(state)
decoder.eval()

# 4) Reconstruct via the full decoder
with torch.no_grad():
    proj_tensor = torch.from_numpy(proj).float()         # (N, D_out)
    recon_tensor = decoder(proj_tensor)                  # (N, D_in)
    recon = recon_tensor.numpy()

# 5) Compute MSE & explained variance
mse = np.mean((raw - recon)**2)
sigma2 = np.mean(raw**2)
r2 = 1 - mse / sigma2

print(f"Reconstruction MSE: {mse:.6f}")
print(f"Input variance σ² : {sigma2:.6f}")
print(f"Explained variance: {r2:.2%}")

Reconstruction MSE: 0.024821
Input variance σ² : 0.140893
Explained variance: 82.38%


can try to imporve by using 
1.	Longer training: run for more epochs or lower the learning rate.
2.	Larger hidden layer: increase H from 512 → 768 or 1024.
3.	Different architecture: add another nonlinearity or use a deeper autoencoder.
4.	Contrastive / supervised tuning: if you have downstream labels (“these champions should match”), fine-tune the encoder directly on that objective rather than pure MSE.